In [5]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import get_sun
import time
from astropy.time import Time
from astropy.coordinates import FK5
from astropy.coordinates import SkyCoord
import random

In [20]:
#Get declination and right ascension of the sun at a given time
time1 = Time('2022-9-25 12:00:00') #The time
sun = get_sun(time1)

sun_coord = SkyCoord(ra=sun.ra.degree*u.degree, dec=sun.dec.degree*u.degree, frame='icrs') #transform the sun coordinates to FK5
sun_coord.galactic
sun_coord.transform_to(FK5(equinox='J2022.9'))

print(sun_coord.dec)

-0d49m52.12601791s


In [ ]:
#mean and standard deviation of the measured lengths of the shadow and stick 

#measurments of the shadow and stick
sigma = 0.25 #cm
shadow_lengths = [20.2, 21.1, 20.6, 20.5, 20.5, 20.9] #cm 
n_shadow = len(shadow_lengths); 

stick_error = 0.25 #cm
stick_lengths = [26.6, 26.7, 26.5, 26.6, 26.6, 26.5]
n_stick = len(stick_lengths)

#calculate the mean and standard deviation of the shadow and stick
shadow_mean = np.mean(shadow_lengths)
shadow_std = np.std(shadow_lengths)
stick_mean = np.mean(stick_lengths)
stick_std = np.std(stick_lengths)

# Stick plot
plt.ion
plt.figure(3)
plt.clf()
plt.errorbar(np.arange(n_stick)+1,stick_lengths,yerr=np.ones(n_stick)*sigma,fmt='o',color='black')
plt.axhline(y=stick_mean,linestyle='--',color='red')
plt.title(str(n_stick)+' "measurements" with mean of '+str(stick_mean)+' and error of '+str(sigma))
plt.xlabel('Measurement Number')
plt.ylabel('Value and error')
plt.title('Stick Lengths')

# Shadow plot
plt.figure(4)
plt.clf()
plt.errorbar(np.arange(n_shadow)+1,shadow_lengths,yerr=np.ones(n_shadow)*sigma,fmt='o',color='black')
plt.axhline(y=shadow_mean,linestyle='--',color='red')
plt.title(str(n_shadow)+' "measurements" with mean of '+str(shadow_mean)+' and error of '+str(sigma))
plt.xlabel('Measurement Number')
plt.ylabel('Value and error')
plt.title('Shadow Lengths')



In [27]:
#Calculate the latitude of Ojai
true_value = 34.44583333333333

sun_ra = sun_coord.ra.degree #right ascension of the sun in degrees
sun_dec = sun_coord.dec.degree #declination of the sun in degrees



#Calculate the latitude of Ojai
def latitude_calc(shadow_lengths, stick_lengths, sun_ra, sun_dec):
    shadow_lengths = np.array(shadow_lengths)
    stick_lengths = np.array(stick_lengths)
    latitude = (shadow_lengths - stick_lengths)*np.tan(np.radians(sun_dec)) + stick_lengths*np.tan(np.radians(sun_ra))
    return latitude

x = latitude_calc(shadow_lengths, stick_lengths, sun_ra, sun_dec); #calculate the latitude of Ojai
print(x)



[0.9840457  0.97579027 0.9734417  0.97969353 0.97969353 0.96908953]
